# Spam Detector - Classification using PySpark
I tried to use PySpark as another alternative for this project, but my results were not as good as the model without PySpark. Here I should also say that I did not understand PySpark that good, but I tried anyways like we did in our lessons.

In [2]:
#pip install pyspark

## Import Libraries and Load Data

In [1]:
# import libraries
from pyspark import SparkConf, SparkContext 
import collections
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.regression import LinearRegression

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

import pandas as pd

In [2]:
# load data
train=pd.read_csv('/content/train.csv')
labels=pd.read_csv("/content/train_labels.csv")
df=pd.concat([train,labels], axis=1) # combine train and label
df.rename(columns={'0': 'label'}, inplace=True) # rename column from '0' to 'label'
df.head()

,0.097094,1.1133,45.038,0.88184,0.087009,1.041,1.5486,3.498,1.8578,0.0096729,...,3.6654,0.061607,0.0031605,0.036038,0.0845,2.4517,3.3373,0.065201,0.091158,label
0,0.050086,0.11158,94.0800,1.76500,0.089417,4.80470,0.26742,NaN,0.56473,0.035123,...,4.16870,0.075432,0.010869,0.063972,0.079892,1.9795,3.5064,0.072132,0.091950,1
1,0.088447,2.36340,5.0580,0.14436,0.064547,2.44400,4.25450,0.36506,1.86090,0.009759,...,4.56130,0.046505,NaN,0.084066,0.064829,3.3087,2.9969,0.064328,0.036793,0
2,0.772540,0.59469,NaN,0.97515,0.015987,0.52884,1.48840,3.96100,4.80630,0.048617,...,0.12832,0.065028,0.036862,0.010010,0.020709,2.5237,2.1711,0.080865,0.081553,0
3,0.382410,4.81090,1955.1000,0.46050,0.024453,2.02980,3.74030,4.22810,2.42920,0.156830,...,4.37010,1.001100,0.065750,0.043547,0.629430,4.6262,3.1947,NaN,0.187180,1
4,0.081316,4.84150,4.0507,2.48320,0.058990,2.37940,1.61270,2.04220,1.65710,0.039377,...,2.68040,0.076524,0.082756,0.041953,0.018092,3.3041,0.1922,0.032600,0.050172,0


In [3]:
# Get the number of columns
n_columns = df.shape[1]

# Create a dictionary with the old column names as keys and the new column names as values
new_column_names = {old_col: 'col' + str(i) for i, old_col in enumerate(df.columns, start=1)}

# Rename the columns
df = df.rename(columns=new_column_names)

print(df.columns)  # The columns will now be named 'col1', 'col2', 'col3', etc.

Index(['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9',
       'col10',
       ...
       'col92', 'col93', 'col94', 'col95', 'col96', 'col97', 'col98', 'col99',
       'col100', 'col101'],
      dtype='object', length=101)


In [4]:
df = df.rename(columns={'col101': 'label'})

In [5]:
df.head()

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col92,col93,col94,col95,col96,col97,col98,col99,col100,label
0,0.050086,0.11158,94.0800,1.76500,0.089417,4.80470,0.26742,NaN,0.56473,0.035123,...,4.16870,0.075432,0.010869,0.063972,0.079892,1.9795,3.5064,0.072132,0.091950,1
1,0.088447,2.36340,5.0580,0.14436,0.064547,2.44400,4.25450,0.36506,1.86090,0.009759,...,4.56130,0.046505,NaN,0.084066,0.064829,3.3087,2.9969,0.064328,0.036793,0
2,0.772540,0.59469,NaN,0.97515,0.015987,0.52884,1.48840,3.96100,4.80630,0.048617,...,0.12832,0.065028,0.036862,0.010010,0.020709,2.5237,2.1711,0.080865,0.081553,0
3,0.382410,4.81090,1955.1000,0.46050,0.024453,2.02980,3.74030,4.22810,2.42920,0.156830,...,4.37010,1.001100,0.065750,0.043547,0.629430,4.6262,3.1947,NaN,0.187180,1
4,0.081316,4.84150,4.0507,2.48320,0.058990,2.37940,1.61270,2.04220,1.65710,0.039377,...,2.68040,0.076524,0.082756,0.041953,0.018092,3.3041,0.1922,0.032600,0.050172,0


In [6]:
# filling all missing values with their mean, because we only have float type columns
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype == 'float' else x)

In [7]:
df.to_csv(r'/content/spam.csv', index=False) # save df to new spam.csv

In [8]:
df2=pd.read_csv('/content/spam.csv')
df2.head()

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col92,col93,col94,col95,col96,col97,col98,col99,col100,label
0,0.050086,0.11158,94.080000,1.76500,0.089417,4.80470,0.26742,2.550598,0.56473,0.035123,...,4.16870,0.075432,0.010869,0.063972,0.079892,1.9795,3.5064,0.072132,0.091950,1
1,0.088447,2.36340,5.058000,0.14436,0.064547,2.44400,4.25450,0.365060,1.86090,0.009759,...,4.56130,0.046505,0.094279,0.084066,0.064829,3.3087,2.9969,0.064328,0.036793,0
2,0.772540,0.59469,291.018659,0.97515,0.015987,0.52884,1.48840,3.961000,4.80630,0.048617,...,0.12832,0.065028,0.036862,0.010010,0.020709,2.5237,2.1711,0.080865,0.081553,0
3,0.382410,4.81090,1955.100000,0.46050,0.024453,2.02980,3.74030,4.228100,2.42920,0.156830,...,4.37010,1.001100,0.065750,0.043547,0.629430,4.6262,3.1947,0.182338,0.187180,1
4,0.081316,4.84150,4.050700,2.48320,0.058990,2.37940,1.61270,2.042200,1.65710,0.039377,...,2.68040,0.076524,0.082756,0.041953,0.018092,3.3041,0.1922,0.032600,0.050172,0


## Create Spark Session

In [9]:
from pyspark.conf import SparkConf

spark=SparkSession.builder\
    .master("local")\
    .appName("SpamDetector")\
    .getOrCreate()
sc=spark.sparkContext

## Preparing the Data

In [10]:
spark_df = spark.read.csv("/content/spam.csv",header=True,inferSchema=True,sep=",")
#inferSchema = True, helps so that every column is not a string
#header = True, takes the column names into the df, other wise the column names will be c0, c1...

In [11]:
spark_df.show(5)

+--------+-------+------------------+-------+--------+-------+-------+-----------------+-------+---------+-------+------+--------+------+------+-------+---------+------+------+--------+--------+--------+------------------+--------+--------+------+-------+--------+--------+------+--------+--------+--------+---------+-------+------+---------+------------------+------------------+--------+------+-------+--------+-------+------+------------------+--------+-------+--------+--------+-------+-------------------+--------+--------+------------------+------+--------+------+--------+---------+---------+--------+--------+--------+---------+-------+-------+-------+--------+--------+-------+-------+---------+---------+------------------+--------+------------------+-------+--------+--------+-------+--------+-------+--------+------------------+------+-------+--------+--------+--------+--------+-------+--------+-------------------+--------+--------+------+------+-------------------+--------+-----+
|   

In [12]:
spark_df.printSchema()

root
 |-- col1: double (nullable = true)
 |-- col2: double (nullable = true)
 |-- col3: double (nullable = true)
 |-- col4: double (nullable = true)
 |-- col5: double (nullable = true)
 |-- col6: double (nullable = true)
 |-- col7: double (nullable = true)
 |-- col8: double (nullable = true)
 |-- col9: double (nullable = true)
 |-- col10: double (nullable = true)
 |-- col11: double (nullable = true)
 |-- col12: double (nullable = true)
 |-- col13: double (nullable = true)
 |-- col14: double (nullable = true)
 |-- col15: double (nullable = true)
 |-- col16: double (nullable = true)
 |-- col17: double (nullable = true)
 |-- col18: double (nullable = true)
 |-- col19: double (nullable = true)
 |-- col20: double (nullable = true)
 |-- col21: double (nullable = true)
 |-- col22: double (nullable = true)
 |-- col23: double (nullable = true)
 |-- col24: double (nullable = true)
 |-- col25: double (nullable = true)
 |-- col26: double (nullable = true)
 |-- col27: double (nullable = true)
 |-- 

In [13]:
spark_df.describe().show()

+-------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------

In [14]:
spark_df.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  253|
|    0|  346|
+-----+-----+



## Vector Assembler
To combine all feature data and separate 'label' data in a dataset, we use VectorAssembler. Then we split our data into train and test data.

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
# I don't like this code, but this was a way for me to do it in a fast way
vect = VectorAssembler(inputCols=['col1',
 'col2',
 'col3',
 'col4',
 'col5',
 'col6',
 'col7',
 'col8',
 'col9',
 'col10',
 'col11',
 'col12',
 'col13',
 'col14',
 'col15',
 'col16',
 'col17',
 'col18',
 'col19',
 'col20',
 'col21',
 'col22',
 'col23',
 'col24',
 'col25',
 'col26',
 'col27',
 'col28',
 'col29',
 'col30',
 'col31',
 'col32',
 'col33',
 'col34',
 'col35',
 'col36',
 'col37',
 'col38',
 'col39',
 'col40',
 'col41',
 'col42',
 'col43',
 'col44',
 'col45',
 'col46',
 'col47',
 'col48',
 'col49',
 'col50',
 'col51',
 'col52',
 'col53',
 'col54',
 'col55',
 'col56',
 'col57',
 'col58',
 'col59',
 'col60',
 'col61',
 'col62',
 'col63',
 'col64',
 'col65',
 'col66',
 'col67',
 'col68',
 'col69',
 'col70',
 'col71',
 'col72',
 'col73',
 'col74',
 'col75',
 'col76',
 'col77',
 'col78',
 'col79',
 'col80',
 'col81',
 'col82',
 'col83',
 'col84',
 'col85',
 'col86',
 'col87',
 'col88',
 'col89',
 'col90',
 'col91',
 'col92',
 'col93',
 'col94',
 'col95',
 'col96',
 'col97',
 'col98',
 'col99',
 'col100'],outputCol='features')

In [17]:
vect_df = vect.transform(spark_df) # transform our DataFrame to a vector
vect_df = vect_df.select(['features', 'label']) # we only select 'features' and 'label' columns because Pyspark ML models wants the dataset like this
vect_df.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.050086,0.11158...|    1|
|[0.088447,2.3634,...|    0|
|[0.77254,0.59469,...|    0|
+--------------------+-----+
only showing top 3 rows



In [19]:
# split data into train and test 
(train, test) = vect_df.randomSplit([0.9, 0.1])

In [20]:
train.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[4.455E-4,0.83678...|    0|
|[4.5758E-4,0.8979...|    0|
|[0.0010957,3.3082...|    1|
+--------------------+-----+
only showing top 3 rows



In [21]:
test.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0032418,3.4921...|    1|
|[0.0036366,1.7777...|    0|
|[0.0053045,2.1663...|    0|
+--------------------+-----+
only showing top 3 rows



## Using Gradient-boosted Tree Classifier to fit and predict our model

In [22]:
# fit the model on train data
gbtc = GBTClassifier(labelCol="label", maxIter=20) #max itteration 20
gbtc = gbtc.fit(train) # fit train data

# predict
pred = gbtc.transform(test) #transform() method is for predicting instead of predict()

In [23]:
pred.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0032418,3.4921...|    1|[-1.5415616507961...|[0.04380879501588...|       1.0|
|[0.0036366,1.7777...|    0|[1.49500622787295...|[0.95212087796977...|       0.0|
|[0.0053045,2.1663...|    0|[1.15434668584643...|[0.90959446563872...|       0.0|
|[0.007249,3.8778,...|    0|[0.82221465492210...|[0.83813673364775...|       0.0|
|[0.0091645,0.5926...|    1|[-1.4284818186251...|[0.05432247219937...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



## Check the accuracy using MulticlassClassificationEvaluator

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix

In [25]:
evaluator=MulticlassClassificationEvaluator(metricName="accuracy")

In [26]:
acc = evaluator.evaluate(pred)
print("Accuracy Score: ", acc)

Accuracy Score:  0.8769230769230769


Accuracy of 87,69% for MultiClassClassificationEvaluator using GBTC.

### Create a confusion matrix

In [27]:
y_predict=pred.select("prediction").collect()
y_original=pred.select("label").collect()
confmatrix = confusion_matrix(y_original, y_predict)

In [28]:
print("Confusion Matrix:")
print(confmatrix)

Confusion Matrix:
[[31  5]
 [ 3 26]]


## Linear Regression with Spark

In [29]:
from pyspark.ml.regression import LinearRegression

In [30]:
#Create Linear regression model
lir = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Train the model using our training data
model = lir.fit(train)

# Now see if we can predict values in our test data.
# Generate predictions using our linear regression model for all features in our
# test dataframe:
fullPredictions = model.transform(test).cache()

# Extract the predictions and the "known" correct labels.
predictions = fullPredictions.select("prediction").rdd.map(lambda x: x[0])
labels = fullPredictions.select("label").rdd.map(lambda x: x[0])

# Zip them together
predictionAndLabel = predictions.zip(labels).collect()

# Print out the predicted and actual values for each point
for prediction in predictionAndLabel:
  print(prediction)

(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 1)
(0.41947565543071164, 1)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 0)
(0.41947565543071164, 1)


## Logistic Regression with Spark

In [31]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [32]:
# Create Logistic Regression model 
lg = LogisticRegression(labelCol='label')
lg_model = lg.fit(train) #fit the model
training_sum = lg_model.summary
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              label|         prediction|
+-------+-------------------+-------------------+
|  count|                534|                534|
|   mean|0.41947565543071164|0.41947565543071164|
| stddev| 0.4939359347743282| 0.4939359347743282|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [33]:
preds = lg_model.evaluate(test)
preds.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0032418,3.4921...|    1|[218.054955002582...|           [1.0,0.0]|       0.0|
|[0.0036366,1.7777...|    0|[3016.53106301712...|           [1.0,0.0]|       0.0|
|[0.0053045,2.1663...|    0|[5171.87240945797...|           [1.0,0.0]|       0.0|
|[0.007249,3.8778,...|    0|[479.216748270881...|           [1.0,0.0]|       0.0|
|[0.0091645,0.5926...|    1|[-341.45440218498...|[5.10784029044185...|       1.0|
|[0.0092304,0.5688...|    0|[6998.16245463894...|           [1.0,0.0]|       0.0|
|[0.011877,4.9623,...|    0|[6566.60265352794...|           [1.0,0.0]|       0.0|
|[0.016221,3.265,1...|    0|[3859.64836871875...|           [1.0,0.0]|       0.0|
|[0.016724,2.8393,...|    1|[-2115.5361569576...|           [0.0,1.0]|       1.0|
|[0.030123,1.354

In [34]:
# Using AUC (Area Under Curve)
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
acc = eval.evaluate(preds.predictions)
print(acc)

0.8371647509578544


Accuracy of 83,72% for Logistic Regression.

## Decision Tree Classifier

In [35]:
from pyspark.ml.classification import DecisionTreeClassifier

In [36]:
# Create a DecisionTreeClassifier and fit the training data
dtc = DecisionTreeClassifier()
dtc_model = dtc.fit(train)

# Create predictions for the testing data and take a look at the predictions
prediction = dtc_model.transform(test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|1    |1.0       |[0.03773584905660377,0.9622641509433962]|
|0    |0.0       |[0.9120879120879121,0.08791208791208792]|
|0    |0.0       |[0.9120879120879121,0.08791208791208792]|
|0    |0.0       |[0.9120879120879121,0.08791208791208792]|
|1    |1.0       |[0.03773584905660377,0.9622641509433962]|
+-----+----------+----------------------------------------+
only showing top 5 rows



In [37]:
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count() #True Negatives
TP = prediction.filter('prediction = 1 AND label = prediction').count() #True Positives
FN = prediction.filter('prediction = 0 AND label = 1').count() #False Negatives
FP = prediction.filter('prediction = 1 AND label = 0').count() #False Psotives

# Accuracy measures the proportion of correct predictions
acc = (TN + TP) / (TN + TP + FN + FP)
print(acc)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|    3|
|    0|       0.0|   31|
|    1|       1.0|   26|
|    0|       1.0|    5|
+-----+----------+-----+

0.8769230769230769


Accuracy of 87,69% for Decision Tree Classifier.

In [38]:
sc.stop() # stop session

As I mentioned in the beginning, I just wanted to try to experiment with PySpark again because I didn't understood it that well in the lessons. The results are not as good as the model without PySpark but I would have to make more research to understand how to improve a PySpark ML model.